In [2]:
import pyspark

conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession(sc)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1b58475d-6e71-4c08-b877-ac78bc2c94e2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.2 in central
	found org.apache

In [1]:
!pip install koalas==1.8.1

In [3]:
import time
OBJECTURL_TEST = 's3a://test-container/playground/colors-test' + str(time.time()) + '.csv'
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

21/10/13 07:17:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
21/10/13 07:17:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+--------+-----+
|    name|color|
+--------+-----+
|   Luigi|Green|
|Princess| Pink|
|   Mario|  Red|
+--------+-----+



In [ ]:
DELTA_URL='s3a://test-container/playground/delts-colors-test' + str(time.time())
gnames_df.write.format("delta").save(DELTA_URL)

In [ ]:
data = spark.range(0, 5)
#data.write.format("delta").save("/tmp/delta-table")

In [ ]:
data.write.format("delta").save("s3a://test-container/playground/aaaaa1")

In [5]:
from databricks import koalas as ks
import pandas as pd
import numpy as np

In [6]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [9]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [7]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [8]:
kdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [12]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
kdf = ks.from_pandas(pdf)
print(type(kdf))
kdf

<class 'databricks.koalas.frame.DataFrame'>


,A,B,C,D
2013-01-01,-1.093838,-0.194637,0.525511,0.507618
2013-01-02,0.886728,0.975830,-0.941501,-0.215265
2013-01-03,-0.287904,-1.098337,1.638812,0.677777
2013-01-04,-0.104464,0.414222,-0.135704,0.218692
2013-01-05,0.075820,0.750705,-0.477105,-0.757437
2013-01-06,2.316590,-0.063603,0.056939,0.423893


In [13]:
sdf = spark.createDataFrame(pdf)
sdf.show()

+--------------------+--------------------+--------------------+--------------------+
|                   A|                   B|                   C|                   D|
+--------------------+--------------------+--------------------+--------------------+
| -1.0938379494214592|-0.19463667110310645|  0.5255106130729222|  0.5076182984120065|
|  0.8867276155904255|  0.9758304851110158|  -0.941501354437414|-0.21526545370080827|
| -0.2879039459879555| -1.0983366452732122|  1.6388124031552997|   0.677776769862656|
|-0.10446368072020998|  0.4142220603553137|-0.13570374148947317| 0.21869162534940312|
| 0.07581983802477416|  0.7507045904714446| -0.4771052126000364| -0.7574367041787883|
|   2.316590076958195|-0.06360302398296386| 0.05693940422490487|  0.4238931835497682|
+--------------------+--------------------+--------------------+--------------------+



In [14]:
kdf = sdf.to_koalas()

In [15]:
sdf.collect()

[Row(A=-1.0938379494214592, B=-0.19463667110310645, C=0.5255106130729222, D=0.5076182984120065),
 Row(A=0.8867276155904255, B=0.9758304851110158, C=-0.941501354437414, D=-0.21526545370080827),
 Row(A=-0.2879039459879555, B=-1.0983366452732122, C=1.6388124031552997, D=0.677776769862656),
 Row(A=-0.10446368072020998, B=0.4142220603553137, C=-0.13570374148947317, D=0.21869162534940312),
 Row(A=0.07581983802477416, B=0.7507045904714446, C=-0.4771052126000364, D=-0.7574367041787883),
 Row(A=2.316590076958195, B=-0.06360302398296386, C=0.05693940422490487, D=0.4238931835497682)]

In [16]:
kdf.to_parquet("s3a://test-container/playground/kdf")

In [17]:
ks.read_parquet("s3a://test-container/playground/kdf").head(10)

21/10/13 07:35:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,-1.093838,-0.194637,0.525511,0.507618
1,0.886728,0.975830,-0.941501,-0.215265
2,-0.287904,-1.098337,1.638812,0.677777
3,-0.104464,0.414222,-0.135704,0.218692
4,0.075820,0.750705,-0.477105,-0.757437
5,2.316590,-0.063603,0.056939,0.423893


In [18]:
kdf.to_delta("s3a://test-container/playground/delta")

In [19]:
ks.read_delta("s3a://test-container/playground/delta").head(10)

21/10/13 07:36:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,-0.104464,0.414222,-0.135704,0.218692
1,0.075820,0.750705,-0.477105,-0.757437
2,2.316590,-0.063603,0.056939,0.423893
3,-1.093838,-0.194637,0.525511,0.507618
4,0.886728,0.975830,-0.941501,-0.215265
5,-0.287904,-1.098337,1.638812,0.677777
